In [ ]:
import os
import json

#Loading Math Vista Dataset
from datasets import load_dataset
from openai import OpenAI # type: ignore

os.environ["OPENAI_API_KEY"] = ""
dataset = load_dataset("AI4Math/MathVista")
client = OpenAI()

In [3]:
# print the first example on the testmini set
# print(dataset["testmini"][13])
# print(dataset["testmini"][13]['pid']) # print the problem id 
# print(dataset["testmini"][13]['question']) # print the question text 
# print(dataset["testmini"][13]['query']) # print the query text
# print(dataset["testmini"][13]['image']) # print the image path
# print(dataset["testmini"][13]['answer']) # print the answer
# dataset["testmini"][13]['decoded_image'] # display the image

In [4]:
answer_type = {'float':0,'integer':0,'list':0,'text':0}

for i in range(1000):
    answer_type[dataset["testmini"][i]['answer_type']] += 1
        

In [5]:
answer_type

{'float': 40, 'integer': 418, 'list': 2, 'text': 540}

In [6]:
int_indices = []
for i in range(1000):
    if dataset["testmini"][i]['answer_type'] == 'integer':
        int_indices.append(i)

In [7]:
responses = []
for index in int_indices:
    
    paraphrase ={
                'pid':None,
                'question':'',
                'query': ''
                }
    
    chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
        "role": "system",
        "content": "You will be provided with math question, and your task is to paraphrase them to standard English."
        },
        {
        "role": "user",
        "content": dataset["testmini"][index]['question']
        }
    ]
    )

    pid = dataset["testmini"][index]['pid']
    response_text = chat_completion.choices[0].message.content
    paraphrase["pid"] = pid
    paraphrase["question"] = response_text
    responses.append(paraphrase)

    

In [8]:
responses_index = 0

for index in int_indices:
    chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
        "role": "system",
        "content": "You will be provided with math question's hint, and your task is to paraphrase them to standard English."
        },
        {
        "role": "user",
        "content": dataset["testmini"][index]['query']
        }
    ]
    )

    response_text = chat_completion.choices[0].message.content

    responses[responses_index]['query'] = response_text
    responses_index += 1

In [9]:


# File name
file_name = "paraphrase_dataset.json"

# Writing the list to a JSON file
with open(file_name, "w") as json_file:
    json.dump(responses, json_file, indent=4)  # 'indent=4' makes the JSON file more readable

print(f"Pharaphrased dataset has been written to {file_name}")


Pharaphrased dataset has been written to paraphrase_dataset.json
